In [1]:
from datasets import load_dataset
DATASET_NAME='csqa'
dataset = load_dataset("tau/commonsense_qa", "", split=["train", "validation"])

In [2]:
NUM_PERMUTE_QUESTION=10
EACH_HAS=50

In [3]:
choice_len=max([len(x['label']) for x in dataset[1]['choices']])
print('max',choice_len)
print(min([len(x['label']) for x in dataset[1]['choices']]))

max 5
5


In [4]:
dataset[1][0]

{'id': '1afa02df02c908a558b4036e80242fac',
 'question': 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?',
 'question_concept': 'revolving door',
 'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
  'text': ['bank', 'library', 'department store', 'mall', 'new york']},
 'answerKey': 'A'}

In [5]:
def get_prompt(data,ques_index, perms, has_choice=False):
    try:
        json_line=data[ques_index]
        question = json_line["question"]
        choices = json_line["choices"]
        choice_texts = choices["text"]
        choice_texts = [choice_texts[perms[i]] for i in range(len(choice_texts))]
        candidates = " ".join(
            [
                f"({label if has_choice else ' '}) {text}"
                for text, label in zip(choice_texts, choices["label"])
            ]
        ).replace("\n", " ")
        answer_key = json_line["answerKey"][0]
        answer_key_idx = ord(answer_key) - (ord("A") if answer_key in 'ABCDE' else ord("1"))
        answer_text = choices["text"][answer_key_idx]
        fact=f"{json_line['fact1']}. " if 'fact1' in json_line else ''
        prompt = f"{fact}{question} \\n {candidates}"
        return prompt, answer_text
    except:
        print(answer_key)

In [6]:
import random
from tqdm import tqdm, trange
import pickle
import itertools
all_permutes=list(itertools.permutations(list(range(choice_len))))
if NUM_PERMUTE_QUESTION:
    container=[]
    for ques_index in trange(NUM_PERMUTE_QUESTION):
        selection=random.sample(all_permutes,EACH_HAS)
        for perm in tqdm(selection):
            container.append(
                    get_prompt(dataset[1],ques_index, perm,  has_choice=False)
                )
    pickle.dump((NUM_PERMUTE_QUESTION, EACH_HAS,container), open(f"{DATASET_NAME}_test_permute.pkl", "wb"))
else:
    container=[get_prompt(x, has_choice=False) for x in tqdm(dataset[0])]
    pickle.dump(container, open(f"{DATASET_NAME}_train.pkl", "wb"))
    pickle.dump([get_prompt(x, has_choice=False) for x in tqdm(dataset[1])], open(f"{DATASET_NAME}_test.pkl", "wb"))

100%|██████████| 10/10 [00:00<00:00, 88.85it/s]
